In [1]:
import numpy as np

In [4]:
%load_ext cython

In [ ]:
%%cython -f --compile-args=-fopenmp --link-args=-fopenmp
# cython: profile=True, cdivision=True, boundscheck=False, wraparound=False
from cython cimport floating

DEF CHUNK_SIZE 256

# TODO make it adaptive
DEF N_THREADS 4

def _pairwise_distance(
    np.ndarray[floating, ndim=2, mode='c'] X,
    np.ndarray[floating, ndim=2, mode='c'] Y,
    # TODO: introduce reduce_func
    # TODO: introduce metric
):
    cdef:
        int X_n = X.shape[0]
        int X_n_features = X.shape[1]
        
        int X_n_samples = X_n * X_n_features
        int X_n_samples_chunk = CHUNK_SIZE if X_n_samples > CHUNK_SIZE else X_n_samples
        int X_n_chunks = X_n_samples // X_n_samples_chunk
        int X_n_samples_rem = X_n_samples % X_n_samples_chunk
        int X_chunk_idx, X_n_samples_chunk_eff
        int X_start, X_end

        int Y_n = Y.shape[0]
        int Y_n_features = Y.shape[1]

        int Y_n_samples = Y_n * Y_n_features
        int Y_n_samples_chunk = CHUNK_SIZE if Y_n_features > CHUNK_SIZE else Y_n_samples
        int Y_n_chunks = Y_n_features // Y_n_samples_chunk
        int Y_n_samples_rem = Y_n_features % Y_n_samples_chunk
        int Y_chunk_idx, Y_n_samples_chunk_eff
        int Y_start, Y_end

        int n_chunks = X_n_samples_chunk * Y_n_samples_chunk
        int n_samples = X_n_samples * Y_n_samples
        
        floating *pairwise_distances_chunk
        
    # count remainder chunk in total number of chunks
    X_n_chunks += (X_n_samples != (X_n_chunks * X_n_samples_chunk))
    Y_n_chunks += (Y_n_samples != (Y_n_chunks * Y_n_samples_chunk))

    # number of threads should not be bigger than number of chunks
    n_threads = min(N_THREADS, n_chunks)

    pairwise_distances = <floating*> malloc(n_samples * sizeof(floating))
    
    with nogil, parallel(num_threads=n_threads):
        # thread local buffers
        
        for X_chunk_idx in prange(X_n_chunks, schedule='static'):
            X_start = X_chunk_idx * X_n_samples_chunk
            if X_chunk_idx == X_n_chunks - 1 and X_n_samples_rem > 0:
                X_end = X_start + X_n_samples_rem
            else:
                X_end = X_start + X_n_samples_chunk
                
            for Y_chunk_idx in prange(Y_n_chunks, schedule='static'):
                Y_start = chunk_idx * Y_n_samples_chunk
                if Y_chunk_idx == Y_n_chunks - 1 and Y_n_samples_rem > 0:
                    Y_end = Y_start + Y_n_samples_rem
                else:
                    Y_end = Y_start + Y_n_samples_chunk

            # TODO: finish implementing
            _pairwise_chunked(
                &X[X_start, X_end],
                &Y[Y_start: Y_end],
                pairwise_distances + X_start * Y_n_samples + Y_start
            )

    